In [38]:
import pandas as pd
import numpy as np

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# Format data to get a uniform style

# Data from https://everytownresearch.org/school-shootings/

In [66]:
everytown = pd.read_csv("./data/everytown.csv",sep=";",header=None, parse_dates=[1])
everytown.columns = ["index","date","date_repeaded","state","city","school","type_school","gun_fired","injured","death","attempted_suicide","suicide","number_victims"]
everytown = everytown.loc[:,["date","state","school","type_school","death","suicide","number_victims"]]
everytown.to_csv("./data/everytown_formatted",index=None,sep="\t")
everytown.head(2)


,date,state,school,type_school,death,suicide,number_victims
0,2013-01-08,FL,Apostolic Revival Center Christian School,K-12,1,0,1
1,2013-01-10,CA,Taft Union High School,K-12,0,0,0


# Data from Jim&Alyssa: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5314897/

In [55]:
#NAN value is: stewart country schools (a K-12 school)
d_type2school = {"High": "K-12", "Middle": "K-12", "college": "College", "Elementary": "K-12",np.NaN:"K-12"}

In [68]:
alyssa = pd.read_csv("./data/ShootMiami2.csv",sep=";",header=None,parse_dates=[0])
alyssa.columns = ["date","_1","_2","_3","year","type_shooing","number_victims","school","type_school","city","state","urbanity",
                 "relationship_with_attacker","gender","age","suicide","motives","population_school","lat","lon","_4","_5","_6","_7","_8"]
alyssa = alyssa.loc[:,["date","state","school","type_school","death","suicide","number_victims"]]
alyssa["death"] = (alyssa["number_victims"]>0).astype(int)
#alyssa["suicide"] = alyssa["suicide"].astype(int) 
alyssa["type_school"] = alyssa["type_school"].replace(d_type2school)
alyssa.to_csv("./data/alyssa_formatted",index=None,sep="\t")
alyssa.head(2)

,date,state,school,type_school,death,suicide,number_victims
0,1990-08-27,NV,Eldorado High,K-12,1,0.0,1
1,1991-10-09,NY,James Monroe,K-12,1,0.0,1


# Data from USA Today on mass shootings (http://www.gannett-cdn.com/GDContent/mass-killings/index.html)

In [69]:
usatoday = pd.read_csv("./data/USAToday.csv",sep="\t",header=None,parse_dates=[0])
usatoday.columns = ["date","city","state","type_attack","motives","number_victims"]
usatoday = usatoday.loc[usatoday["type_attack"].str.contains("hooting")]
usatoday.to_csv("./data/usatoday_formatted",index=None,sep="\t")
usatoday.head(2)

,date,city,state,type_attack,motives,number_victims
1,2006-01-30,Goleta,Calif.,Shooting,Public Killing,7
2,2006-02-21,Mesa,Ariz.,Shooting,Other,5
